Ant Colony Optimization (ACO) >> Think of Ants as Agents in a Multi-Agent ENV Setup >> Colaborative


Walkthrough
This notebook demonstrates how Ant Colony Optimization (ACO) can be used by AI agents to navigate a maze. We’ll simulate a group of ants trying to find the path from a start to a goal.

Ants can succeed (find food) or fail (dead end or loop).
Successful ants leave pheromones on their path.
Pheromones evaporate over time, preventing the colony from getting stuck.


In [1]:
import random
import numpy as np

In [2]:
# 0 = free cell, 1 = wall
maze = [
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0]
]



'''
maze = [
    [0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0],
    [0, 0, 1, 0, 0],
]
'''
start, end = (0, 0), (2, 2)   # Start top-left, goal bottom-right

In [3]:
n_ants = 5            # Number of ants per iteration
n_iterations = 2      # Number of rounds
evaporation_rate = 0.5  # Start with 0.1 (slow fade), later try 0.9

# Pheromone levels (all start equal = 1)
pheromones = np.ones((len(maze), len(maze[0])))

# Moves: right, left, down, up,
# Add diagonals, just as we did in the ealier weeks
moves = [(0,1),(0,-1),(1,0),(-1,0)]

In [4]:
# Check if cell is valid (inside maze and not a wall)
def valid(x, y):
    return 0 <= x < len(maze) and 0 <= y < len(maze[0]) and maze[x][y] == 0

# Simulate one ant walking
def ant_walk():
    path = [start]
    visited = set([start])
    current = start

    while current != end:
        x, y = current
        # Possible moves
        neighbors = [(x+dx, y+dy) for dx,dy in moves if valid(x+dx,y+dy)]
        if not neighbors:
            return None  # Dead end → failure

        # Pick next cell with probability ∝ pheromone level
        probs = [pheromones[nx][ny] for nx,ny in neighbors]
        total = sum(probs)
        probs = [p/total for p in probs]
        current = random.choices(neighbors, probs)[0]

        if current in visited:
            return None  # Loop → failure

        visited.add(current)
        path.append(current)

    return path  # Success

In [9]:
best_path = None
for it in range(1, n_iterations+1):
    print(f"\n=== Iteration {it} ===")
    all_paths = []

    # Each ant tries to find the goal
    for a in range(1, n_ants+1):
        path = ant_walk()
        if path:
            print(f"Ant {a}: SUCCESS (length {len(path)}) {path}")
            all_paths.append(path)
            # Deposit pheromone on path (shorter path → more pheromone)
            for (x,y) in path:
                pheromones[x][y] += 1.0 / len(path)
        else:
            print(f"Ant {a}: FAILED (dead end or loop)")

    # Evaporation: reduce pheromone everywhere
    pheromones *= (1 - evaporation_rate)

    # Track best path so far
    if all_paths:
        best = min(all_paths, key=len)
        if best_path is None or len(best) < len(best_path):
            best_path = best
    print(f"Best so far: {len(best_path) if best_path else None} steps")


=== Iteration 1 ===
Ant 1: SUCCESS (length 5) [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]
Ant 2: FAILED (dead end or loop)
Ant 3: FAILED (dead end or loop)
Ant 4: FAILED (dead end or loop)
Ant 5: FAILED (dead end or loop)
Best so far: 5 steps

=== Iteration 2 ===
Ant 1: FAILED (dead end or loop)
Ant 2: FAILED (dead end or loop)
Ant 3: FAILED (dead end or loop)
Ant 4: SUCCESS (length 5) [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]
Ant 5: FAILED (dead end or loop)
Best so far: 5 steps


In [6]:
print("\nFinal Best Path:")
print(best_path)


Final Best Path:
[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)]
